In [2]:
import pandas as pd
import ast
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
from dotenv import load_dotenv
import re

# Load environment variables from .env file
load_dotenv()
GPT_MODEL = "gpt-3.5-turbo-0613"
openai.api_key = os.getenv("OPENAI_API_KEY")

## Extract ingredients from the recipe

In [3]:
# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input: list[str]) -> list[list[str]]:
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]

In [20]:
# recipe = """
# 3 heaping tablespoons of peanut butter
# 3 tablespoons of sambal sauce
# 3 tablespoons of soy sauce
# I tablespoons of honey
# 1 tablespoons of sesame oil
# The juice of 1 lime
# 1 garlic, grated
# 1 small knob of ginger, grated
#  cup of water
# 1 cup of mushrooms
# 1/2 cup of cabbage leaves
# 2 servings of soba noodles
# Chopped scallion
# Chopped cilantro
# """ 
recipe = """
1 pound of boneless chicken thighs, skin on
4 cloves of garlic 
1/4 cup of soy sauce 
Slight less than 1/4 cup of honey 
1 head of broccoli
"""
# Experiment with prompts
# Change plural ingredients to singular. For example, "noodles" to "noodle".

# ChatGPT to help get similar items
QUERIES_INPUT = f"""
Get all the ingredients in the recipe, e.g. flour, egg, milk,... This is the recipe: {recipe}
Remove any basic ingredient like water.

Simplify the ingredients. For example, "cabbage leaves to cabbage".
Format: {{"Products": ["product_1", "product_2",...]}}
"""

similar_products = json_gpt(QUERIES_INPUT)["Products"]
print(similar_products)

['boneless chicken thighs', 'garlic', 'soy sauce', 'honey', 'broccoli']


## Find the right general category

In [21]:
general_categories = ["bakery", "dairy-eggs-fridge", "drinks", "freezer", "fruit-veg", "health-wellnes health-foods", "lunch-box", "pantry", "poultry-meat-seafood"]


In [22]:
category_dict = {
    "bakery": ["bakery", "bread", "pastries"],
    "dairy-eggs-fridge": ["dairy-eggs-fridge", "milk", "cheese", "yogurt", "cream", "dips", "ready to eat meals", "international food", "vegan"],
    "drinks": ["drinks", "juices", "soda", "water", "tea", "coffee", "energy drinks"],
    "freezer": ["freezer", "frozen meals", "ice cream", "frozen vegetables", "frozen fruit"],
    "fruit-veg": ["fruit-veg", "fruits", "vegetables", "salads", "organic", "fresh herbs"],
    "health-wellness health-foods": ["health-wellness health-foods", "vitamins", "superfoods", "protein bars", "health foods", "dried fruit, nuts, seeds"],
    "lunch-box": ["lunch-box","sandwiches", "snack packs", "fruit cups"],
    "pantry": ["pantry", "canned goods", "breakfast and spreads", "herbs and spices", "condiments", "canned food", "pasta, rice, grains", "cooking sauces and recipe bases", "oil and vinegar", "international foods"],
    "poultry-meat-seafood": ["poultry-meat-seafood", "poultry", "meat", "seafood"]
}

In [23]:
category_list = [item for sublist in category_dict.values() for item in sublist]

In [24]:
# ChatGPT to help get similar items
QUERIES_INPUT = f"""
Group all the items into the right categories. 
These are the categories: {category_list}.
These are the items: {similar_products}.
Format: {{"category": ["item_1", "item_2",...]}}
"""

similar_products = json_gpt(QUERIES_INPUT)
print(similar_products)

{'poultry-meat-seafood': ['boneless chicken thighs'], 'herbs and spices': ['garlic'], 'cooking sauces and recipe bases': ['soy sauce'], 'breakfast and spreads': ['honey'], 'vegetables': ['broccoli']}


In [25]:
categorized_items = {}
# Iterate over the items
for key, value in similar_products.items():
    for category, keywords in category_dict.items():
        # Check if any keyword in the category is present in the item
        if key in keywords:
            categorized_items[category] = categorized_items.get(category, []) + value
            break

print(categorized_items)

{'poultry-meat-seafood': ['boneless chicken thighs'], 'pantry': ['garlic', 'soy sauce', 'honey'], 'fruit-veg': ['broccoli']}


## Find the product

In [26]:
# Bad list
bad_list = [
"Artificial flavor",
"Artificial flavour",
"Natural flavor",
"Natural flavour",
"Aspartame",
"BHT",
"Calcium disodium EDTA",
"Caramel color",
"Carrageenan",
"Corn starch",
"Corn syrup",
"Dextrose",
"Dough conditioners",
"Enriched flour",
"Bleached flour",
"Food color",
"Maltodextrin",
"Monoglycerides",
"Monosodium glutamate",
"Diglyceride",
"Natural flavor",
"Natural flavors",
"Polysorbate",
"Potassium sorbate",
"Sodium erythorbate",
"Sodium nitrate",
"Sodium nitrite",
"Sodium phosphate",
"Soy protein isolate",
"Splenda",
"Sugar",
"Syrup",
"Skim milk",
"Low fat",
"Reduced fat",
"Xylitol",
]

In [27]:
def find_product(product, df ,k):
    product_split = product.split()

    # ChatGPT to help get similar items
    QUERIES_INPUT = f"""
    Give me similar products related to this prompt but is not it: {product}
    ONLY if the items are similar, otherwise don't.
    Example: if the prompt is milk then similar products would be: cheese, butter, yoghurt, etc.
    Include variations of the product name, e.g. yogurt and yoghurt
    Format: {{"Products": ["product_1", "product_2",...]}}
    """

    similar_products = json_gpt(QUERIES_INPUT)["Products"]

    # Filter out rows that do not contain the product name
    selected_rows = df.copy()  # Create a copy of the original dataframe

    for keyword in product_split:
        selected_rows = selected_rows[selected_rows['Product Name'].str.contains(fr'\b{re.escape(keyword)}\b', case=False)]
    for item in similar_products:
        selected_rows = selected_rows[~selected_rows['Product Name'].str.contains(fr'\b{re.escape(item)}\b', case=False)]
    
    # HARD CODE FILTERING

    # Filter out rows with no ingredients for certain categories only
    if k != 'fruit-veg' and k != 'poultry-meat-seedfood':
        selected_rows = selected_rows[~selected_rows['Ingredients'].isna()]
    if product == "honey":
        selected_rows = selected_rows[selected_rows["Aisle"].str.lower() == "honey"]
    if product == "ginger":
        selected_rows = selected_rows[selected_rows["Department"].str.lower() != "drink"]
        

    



    # Get the 'Product Name' and 'Ingredients' columns as Series
    product_names = selected_rows['Product Name']
    ingredients_series = selected_rows['Ingredients']
    cup_prices = selected_rows['Cup Price']
    price = selected_rows['Price']

    clean_products_df = pd.DataFrame(columns=['Product Name', 'Ingredients', 'Cup Price', 'Price'])
    all_clean = []

    # Iterate over each ingredient string along with its corresponding product name and cup price
    for product_name, ingredients, cup_price, price in zip(product_names, ingredients_series, cup_prices, price):
        clean = True
        # For categories like fruit-veg or poultry-meat-seafood, the ingredients list is empty -> if isinstance
        # Split the string at commas that are not between parentheses
        if isinstance(ingredients, str):
            ingredients_list = re.split(r',\s*(?![^()]*\))', ingredients)
        # else:
        #     ingredients_list = []
        
            # Iterate over each ingredient in the list
            for ingredient in ingredients_list:
                # Check if the ingredient is in the bad_list
                for bad_item in bad_list:
                    # Normalize bad_list item to lowercase and split it into individual words
                    bad_item_lower = bad_item.lower()
                    bad_words = re.findall(r'\b\w+\b', bad_item_lower)
                    
                    # Check if all the words from bad_list are present in the ingredient
                    all_words_present = all(word in ingredient.lower() for word in bad_words)
                    
                    if all_words_present:
                        clean = False
        else:
            ingredients_list = []

        # Ingredients shouldn't be more than a certain amount
        gum = 0
        oil = 0
        emulsifier = 0
        # Count the occurrences of specific ingredients
        gum = sum(ingredient.lower().count("gum") for ingredient in ingredients_list)
        oil = sum(ingredient.lower().count("oil") for ingredient in ingredients_list)
        emulsifier = sum(ingredient.lower().count("emulsifier") for ingredient in ingredients_list)

        if gum > 2 or oil > 2 or emulsifier > 2:
            clean = False

        # If the product is clean, add it to the list
        if clean:
            clean_products_df = pd.concat([clean_products_df, pd.DataFrame({
                'Product Name': [product_name],
                'Ingredients': [ingredients],
                'Cup Price': [cup_price],
                "Price": [price] 
            })])
    clean_products_df_sorted = clean_products_df.sort_values(by='Cup Price')
    return clean_products_df_sorted

In [29]:
grocery_list = pd.DataFrame(columns=['Product Name', 'Ingredients', 'Cup Price', 'Price'])
all_none = []

for k, v in categorized_items.items():
    # Load files
    # Because there are 2 files for pantry items
    file_path2 = None
    if k == "pantry":
        file_path = "Data\Woolies Extracted\Woolies {} 1 info.xlsx".format(k)
        file_path2 = "Data\Woolies Extracted\Woolies {} 2 info.xlsx".format(k)
    else:
        file_path = "Data\Woolies Extracted\Woolies {} info.xlsx".format(k)
    df = pd.read_excel(file_path)
    if file_path2:
        df2 = pd.read_excel(file_path2)
        df = pd.concat([df, df2], ignore_index=True)
    # Find product 
    for product in v:
        print(product)
        clean_products_df_sorted = find_product(product, df, k)

        # Find similar products (ex: Spring onion -> green onion) if not found
        if clean_products_df_sorted.empty:
            QUERIES_INPUT = f"""
                Give me the other names of the the product in this prompt: {product}
                ONLY if the names refer to one specific thing, otherwise don't.
                Example: if the prompt is spring onion then similar products would be: green onions, scallions etc.
                Format: ["alternative_name_1", "alternative_name_2",...]
            """
            similar_products = json_gpt(QUERIES_INPUT)
            print("Similar products: ", similar_products)
            for product in similar_products:
                clean_products_df_sorted = find_product(product, df, k)
                if not clean_products_df_sorted.empty:
                    break
                print("Similar: ")
            
        grocery_list = pd.concat([grocery_list, clean_products_df_sorted.head(1)], ignore_index=True)
        print(clean_products_df_sorted.head(1))
        
        if clean_products_df_sorted.empty:
            all_none.append(product)

print(all_none)



boneless chicken thighs
Similar products:  ['boneless chicken legs', 'skinless chicken thighs']
Similar: 
Similar: 
Empty DataFrame
Columns: [Product Name, Ingredients, Cup Price, Price]
Index: []
garlic
               Product Name  \
0  Woolworths Minced Garlic   

                                         Ingredients Cup Price Price  
0  Rehydrated Garlic (99%) (Water, Dried Garlic),...      0.05   2.4  
soy sauce
                    Product Name  \
0  Nam Duong Hang Viet Soy Sauce   

                                         Ingredients Cup Price Price  
0  Water, Hydrolysed Vegetable Protein (from Soyb...      0.45  2.25  
honey
                      Product Name Ingredients Cup Price Price
0  Cloverdale Pure Honey Handypack       Honey      0.92  23.0
broccoli
     Product Name Ingredients Cup Price Price
0  Fresh Broccoli         NaN      1.29  1.29
['skinless chicken thighs']


In [30]:
grocery_list

,Product Name,Ingredients,Cup Price,Price
0,Woolworths Minced Garlic,"Rehydrated Garlic (99%) (Water, Dried Garlic),...",0.05,2.4
1,Nam Duong Hang Viet Soy Sauce,"Water, Hydrolysed Vegetable Protein (from Soyb...",0.45,2.25
2,Cloverdale Pure Honey Handypack,Honey,0.92,23.0
3,Fresh Broccoli,NaN,1.29,1.29


In [31]:
total_cost = grocery_list["Price"].sum()

print("Total Cost: $" + str(total_cost))

Total Cost: $28.939999999999998


In [12]:

# QUERIES_INPUT = f"""
# You have access to a list of Sap Categories: {df["Sap Category Name"].unique()}
# You also have access to a list of Sap Sub Categories: {df["Sap Sub Category Name"].unique()}
# Return all of the relevant categories within the list for the following item: {product}
# ONLY return the categories within the list provided.
# Format: {{"Sap Category": ["category_1", "category_2",...], "Sap Sub Category": ["category_1", "category_2",...]}}
# """

# res_sap = json_gpt(QUERIES_INPUT)["Sap Category"]
# # res_sub_sap = json_gpt(QUERIES_INPUT)["Sap Sub Category"]
# # res_sap, res_sub_sap
# json_gpt(QUERIES_INPUT)

## Debug

In [46]:
df = pd.read_excel("Data\Woolies Extracted\Woolies pantry 1 info.xlsx")

df2 = pd.read_excel("Data\Woolies Extracted\Woolies pantry 2 info.xlsx")
df = pd.concat([df, df2], ignore_index=True)

In [47]:
# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input: list[str]) -> list[list[str]]:
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]

bad_list = [
    "Artificial flavor",
    "Artificial flavour",
    "Natural flavor",
    "Natural flavour",
    "Aspartame",
    "BHT",
    "Calcium disodium EDTA",
    "Caramel color",
    "Carrageenan",
    "Corn starch",
    "Corn syrup",
    "Dextrose",
    "Dough conditioners",
    "Enriched flour",
    "Bleached flour",
    "Food color",
    "Maltodextrin",
    "Monoglycerides",
    "Monosodium glutamate",
    "Diglyceride",
    "Natural flavor",
    "Natural flavors",
    "Polysorbate",
    "Potassium sorbate",
    "Sodium erythorbate",
    "Sodium nitrate",
    "Sodium nitrite",
    "Sodium phosphate",
    "Soy protein isolate",
    "Splenda",
    "Sugar",
    "Syrup",
    "Skim milk",
    "Low fat",
    "Reduced fat",
    "Xylitol",
]

product = "soba noodle"

product_split = product.split()

# ChatGPT to help get similar items
QUERIES_INPUT = f"""
Give me similar products related to this prompt but is not it: {product}
ONLY if the items are similar, otherwise don't.
Example: if the prompt is milk then similar products would be: cheese, butter, yoghurt, etc.
Include variations of the product name, e.g. yogurt and yoghurt
Format: {{"Products": ["product_1", "product_2",...]}}
"""

similar_products = json_gpt(QUERIES_INPUT)["Products"]
print("Similar products: ", similar_products)

# Filter out rows that do not contain the product name
selected_rows = df.copy()  # Create a copy of the original dataframe
for keyword in product_split:
    selected_rows = selected_rows[selected_rows['Product Name'].str.contains(fr'\b{re.escape(keyword)}\b', case=False)]
for item in similar_products:
    selected_rows = selected_rows[~selected_rows['Product Name'].str.contains(fr'\b{re.escape(item)}\b', case=False)]
# Filter out rows with no ingredients for certain categories only
k = 'fruit-veg'
if k != 'fruit-veg' and k != 'poultry-meat-seedfood':
    print("here")
    selected_rows = selected_rows[~selected_rows['Ingredients'].isna()]
print(len(selected_rows))

# Get the 'Product Name' and 'Ingredients' columns as Series
product_names = selected_rows['Product Name']
ingredients_series = selected_rows['Ingredients']
cup_prices = selected_rows['Cup Price']
price = selected_rows['Price']

clean_products_df = pd.DataFrame(columns=['Product Name', 'Ingredients', 'Cup Price', 'Price'])
all_clean = []

# Iterate over each ingredient string along with its corresponding product name and cup price
for product_name, ingredients, cup_price, price in zip(product_names, ingredients_series, cup_prices, price):
    clean = True

    # For categories like fruit-veg or poultry-meat-seafood, the ingredients list is empty -> if isinstance
    # Split the string at commas that are not between parentheses
    if isinstance(ingredients, str):
        ingredients_list = re.split(r',\s*(?![^()]*\))', ingredients)
    # else:
    #     ingredients_list = []
    
        # Iterate over each ingredient in the list
        for ingredient in ingredients_list:
            # Check if the ingredient is in the bad_list
            for bad_item in bad_list:
                # Normalize bad_list item to lowercase and split it into individual words
                bad_item_lower = bad_item.lower()
                bad_words = re.findall(r'\b\w+\b', bad_item_lower)
                
                # Check if all the words from bad_list are present in the ingredient
                all_words_present = all(word in ingredient.lower() for word in bad_words)
                
                if all_words_present:
                    clean = False

    # Ingredients shouldn't be more than a certain amount
    gum = 0
    oil = 0
    emulsifier = 0
    # Count the occurrences of specific ingredients
    gum = sum(ingredient.lower().count("gum") for ingredient in ingredients_list)
    oil = sum(ingredient.lower().count("oil") for ingredient in ingredients_list)
    emulsifier = sum(ingredient.lower().count("emulsifier") for ingredient in ingredients_list)

    if gum > 2 or oil > 2 or emulsifier > 2:
        print("Too many")
        clean = False

    # If the product is clean, add it to the list
    if clean:
        clean_products_df = pd.concat([clean_products_df, pd.DataFrame({
            'Product Name': [product_name],
            'Ingredients': [ingredients],
            'Cup Price': [cup_price],
            "Price": [price] 
        })])

clean_products_df_sorted = clean_products_df.sort_values(by='Cup Price')
first_row = clean_products_df_sorted.head(1)

if first_row.empty:
    print("Can't find a product.")
else:
    print("There's a product.")
clean_products_df_sorted.head(1)

Similar products:  ['udon noodle', 'ramen noodle', 'pho noodle', 'rice noodle']
0
Can't find a product.


,Product Name,Ingredients,Cup Price,Price


The DataFrame is empty.


True

In [15]:
# clean_products_df_sorted.to_excel('clean_products.xlsx', index=False)

In [49]:
similar_products = {
    'product1': [1, 2, 3],
    'product2': [4, 5, 6],
    'product3': [7, 8, 9]
}

for key, value in similar_products.items():
    print("Key:", key)
    for item in value:
        if item == 1:  # Replace 'condition' with your desired condition to skip to the next key
            continue
        print(key, value)
        print("Value:", item)

Key: product1
product1 [1, 2, 3]
Value: 2
product1 [1, 2, 3]
Value: 3
Key: product2
product2 [4, 5, 6]
Value: 4
product2 [4, 5, 6]
Value: 5
product2 [4, 5, 6]
Value: 6
Key: product3
product3 [7, 8, 9]
Value: 7
product3 [7, 8, 9]
Value: 8
product3 [7, 8, 9]
Value: 9
